# Testing Dhivehi Pairs

Here we will try to distil sentence transformer performance from a *paraphrase* sentence transformer to a pretrained XLMR model that has *not* been pretrained with Dhivehi text.

First task is to load the Dhivehi pairs.

In [2]:
with open('../data/entodv.tsv', 'r') as fp:
    entodv = fp.readlines()

entodv[0]

'"formed bank to increase competitiveness, reduce interest"\tބޭންކު ހެދީ ވާދަވެރިކަން ބޮޑުކޮށް، އިންޓަރެސްޓް ކުޑަ ކުރަން\n'

In [3]:
entodv[1]

'one of the shareholders of commercial bank of maldives (cbm), champa hussain afeef has stated he started a new bank to increase competitiveness and reduce interest rates in the market. \tރާއްޖޭގައި އިތުރު ބޭންކެއް ހެދުމުގެ މަގުސަދަކީ ބޭންކިން ދާއިރާގެ ވާދަވެރިކަން ބޮޑުކޮށް، އިންޓަރެސްޓް ރޭޓް ކުޑަ ކުރުމަށް ބާރު އަޅަން ކަމަށް ކޮމާޝަލް ބޭންކް އޮފް މޯލްޑިވްސް (ސީބީއެމް)ގެ އެއް ހިއްސާދާރު އަދި މަޝްހޫރު ވިޔަފާރިވެރިޔާ، ޗަމްޕާ ހުސެއިން އަފީފު މިއަދު ވިދާޅުވެއްޖެ އެވެ.\n'

In [4]:
entodv[2]

'in an exclusive interview to avas, champa resorts chairman afeef said that forming a bank was a vision of his brother popular businessman mohamed moosa (uhchu). afeef said that when he first entered the resort market, interest rates were extremely high and therefore negatively impacted on business growth. therefore, he said he had envisioned opening a bank to make the sector more competitive and give easier access to loans and other financial instruments. \tއަވަސް އަށް ދެއްވި ހާއްސަނ އިންޓަވިއެއްގައި ކްރައުން އެންޑް ޗަމްޕާ ރިސޯޓްސްގެ ޗެއާމަން އަފީފު ވިދާޅުވީ ބޭންކެއް ހެދުމުގެ ވިސްނުމަކީ އަފީފްގެ ބޭބެ، މަޝްހޫރު ވިޔަފާރިވެރިޔާ، މުހައްމަދު މޫސާ (އުއްޗު) ގެ ވިސްނުމެއް ކަމަށެވެ. އަފީފް ވިދާޅުވީ އެންމެ ފުރަތަމަ ރިސޯޓް ހަދަން މަސައްކަތް ކުރެއްވި އިރު އޭރު ރާއްޖޭގައި އިންޓަރެސްޓް ރޭޓް އުޅެނީ ވަރަށް ބޮޑު ކަމަށާއި އެކަމުގެ ސަބަބުން ވިޔަފާރި ފުޅާ ކުރަން ލޯނު ނެގުން އޮތީ ދަތިކަމަކަށް ވެފައި ކަމަށެވެ. އަދި ބޭންކެއް ހެދުމުގެ ހުވަފެން އައީ، މި ދާއިރާއަށް ވާދަވެރިކަން ގެނެސްގެން ލޯނު ފަދަ ފައިނޭންސް 

Looks good so far, let's split some of this data to act as our evaluation set later.

In [7]:
from random import sample

dev_split = int(len(entodv) * 0.05)

idx = sample(list(range(len(entodv))), dev_split)

dev_set = [entodv[i] for i in idx]
train_set = [entodv[i] for i in range(len(entodv)) if i not in idx]

print(f"original: {len(entodv)}\ntrain set: {len(train_set)}\ndev set: {len(dev_set)}")

original: 23770
train set: 22582
dev set: 1188


We can go ahead and save these as a .tsv.gz files ready for loading into our `ParallelSentencesDataset` object later.

In [10]:
import gzip
import os

with gzip.open('../data/entodv-train.tsv.gz', 'wt', encoding='utf-8') as fp:
    fp.write('\n'.join(train_set))
with gzip.open('../data/entodv-dev.tsv.gz', 'wt', encoding='utf-8') as fp:
    fp.write('\n'.join(dev_set))

os.listdir('../data')

['entodv-dev.tsv.gz', 'entodv-train.tsv.gz', 'entodv.tsv']

Then we need to confirm that the XLMR tokenizer can deal with Dhivehi characters.

In [11]:
sentences = []

for pair in entodv[:5]:
    dv = pair.split('\t')[1]
    sentences.append(dv)

sentences

['ބޭންކު ހެދީ ވާދަވެރިކަން ބޮޑުކޮށް، އިންޓަރެސްޓް ކުޑަ ކުރަން\n',
 'ރާއްޖޭގައި އިތުރު ބޭންކެއް ހެދުމުގެ މަގުސަދަކީ ބޭންކިން ދާއިރާގެ ވާދަވެރިކަން ބޮޑުކޮށް، އިންޓަރެސްޓް ރޭޓް ކުޑަ ކުރުމަށް ބާރު އަޅަން ކަމަށް ކޮމާޝަލް ބޭންކް އޮފް މޯލްޑިވްސް (ސީބީއެމް)ގެ އެއް ހިއްސާދާރު އަދި މަޝްހޫރު ވިޔަފާރިވެރިޔާ، ޗަމްޕާ ހުސެއިން އަފީފު މިއަދު ވިދާޅުވެއްޖެ އެވެ.\n',
 'އަވަސް އަށް ދެއްވި ހާއްސަނ އިންޓަވިއެއްގައި ކްރައުން އެންޑް ޗަމްޕާ ރިސޯޓްސްގެ ޗެއާމަން އަފީފު ވިދާޅުވީ ބޭންކެއް ހެދުމުގެ ވިސްނުމަކީ އަފީފްގެ ބޭބެ، މަޝްހޫރު ވިޔަފާރިވެރިޔާ، މުހައްމަދު މޫސާ (އުއްޗު) ގެ ވިސްނުމެއް ކަމަށެވެ. އަފީފް ވިދާޅުވީ އެންމެ ފުރަތަމަ ރިސޯޓް ހަދަން މަސައްކަތް ކުރެއްވި އިރު އޭރު ރާއްޖޭގައި އިންޓަރެސްޓް ރޭޓް އުޅެނީ ވަރަށް ބޮޑު ކަމަށާއި އެކަމުގެ ސަބަބުން ވިޔަފާރި ފުޅާ ކުރަން ލޯނު ނެގުން އޮތީ ދަތިކަމަކަށް ވެފައި ކަމަށެވެ. އަދި ބޭންކެއް ހެދުމުގެ ހުވަފެން އައީ، މި ދާއިރާއަށް ވާދަވެރިކަން ގެނެސްގެން ލޯނު ފަދަ ފައިނޭންސް ލިބޭނެ އިންތިޒާމް ހަމަޖެއްސުމުގެ ވިސްނުމުގައި ކަމަށެވެ.\n',
 'ބޭބެ، މުހައްމަދު މޫސާ، އުއްޗު ބޭންކް އޮފް މޯލްޑި

In [12]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

for text in sentences:
    print(tokenizer.tokenize(text))

['▁', 'ބޭންކު', '▁', 'ހެދީ', '▁', 'ވާދަވެރިކަން', '▁', 'ބޮޑުކޮށް', '،', '▁', 'އިންޓަރެސްޓް', '▁', 'ކުޑަ', '▁', 'ކުރަން']
['▁', 'ރާއްޖޭގައި', '▁', 'އިތުރު', '▁', 'ބޭންކެއް', '▁', 'ހެދުމުގެ', '▁', 'މަގުސަދަކީ', '▁', 'ބޭންކިން', '▁', 'ދާއިރާގެ', '▁', 'ވާދަވެރިކަން', '▁', 'ބޮޑުކޮށް', '،', '▁', 'އިންޓަރެސްޓް', '▁', 'ރޭޓް', '▁', 'ކުޑަ', '▁', 'ކުރުމަށް', '▁', 'ބާރު', '▁', 'އަޅަން', '▁', 'ކަމަށް', '▁', 'ކޮމާޝަލް', '▁', 'ބޭންކް', '▁', 'އޮފް', '▁', 'މޯލްޑިވްސް', '▁(', 'ސީބީއެމް', ')', 'ގެ', '▁', 'އެއް', '▁', 'ހިއްސާދާރު', '▁', 'އަދި', '▁', 'މަޝްހޫރު', '▁', 'ވިޔަފާރިވެރިޔާ', '،', '▁', 'ޗަމްޕާ', '▁', 'ހުސެއިން', '▁', 'އަފީފު', '▁', 'މިއަދު', '▁', 'ވިދާޅުވެއްޖެ', '▁', 'އެވެ', '.']
['▁', 'އަވަސް', '▁', 'އަށް', '▁', 'ދެއްވި', '▁', 'ހާއްސަނ', '▁', 'އިންޓަވިއެއްގައި', '▁', 'ކްރައުން', '▁', 'އެންޑް', '▁', 'ޗަމްޕާ', '▁', 'ރިސޯޓްސްގެ', '▁', 'ޗެއާމަން', '▁', 'އަފީފު', '▁', 'ވިދާޅުވީ', '▁', 'ބޭންކެއް', '▁', 'ހެދުމުގެ', '▁', 'ވިސްނުމަކީ', '▁', 'އަފީފްގެ', '▁', 'ބޭބެ', '،', '▁', 'މަޝްހޫރު', '▁', 'ވިޔަފާރިވެރި

I have no idea what any of this means, but this looks like it's splitting into words/subwords, and there are no unknown tokens - so it looks promising. Let's initialize a student and teacher model.

In [13]:
from sentence_transformers import models, SentenceTransformer

xlmr = models.Transformer('xlm-roberta-base')
pooler = models.Pooling(
    xlmr.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

student = SentenceTransformer(modules=[xlmr, pooler])

teacher = SentenceTransformer('paraphrase-distilroberta-base-v2')  # monolingual sentence transformer

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we move onto prepping everything for fine-tuning, start with getting the data ready.

In [14]:
from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(student_model=student, teacher_model=teacher, batch_size=32, use_embedding_cache=True)

Load data...

In [16]:
data.load_data('../data/entodv-train.tsv.gz', max_sentences=50_000, max_sentence_length=256)

In [17]:
from torch.utils.data import DataLoader

loader = DataLoader(data, shuffle=True, batch_size=32)

Now initialize the loss function.

In [18]:
from sentence_transformers import losses

loss = losses.MSELoss(model=student)

And finally we can train...

In [ ]:
epochs = 2
warmup_steps = int(len(loader) * epochs * 0.1)

student.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='../models/xlmr-dv-sentence-vecs',
    optimizer_params={'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False},
    save_best_model=True
)